In [27]:
%run ./spark-instance.ipynb

SparkConf created
Started SparkSession
Spark version 3.3.1


In [63]:
clean_spark()

CLEANING SPARK INSTANCE...


In [16]:
from datetime import datetime
import os

import pyspark.sql.types as pst
import pyspark.sql.functions as psf

In [33]:
base_path = "s3a://censys/dataset=certificates-v2-full/format=parquet/year={year}/month={month:02d}/day={day:02d}"
batch_path = os.path.join(base_path, "batch={batch:03d}")
ss_ts = datetime(2024, 3, 1)
#nr_batches = 18


def binary_to_hex(binary):
    if binary is None:
        return None
    #hexadecimal = hex(binary).lstrip("0x").rstrip("L").lstrip("0")
    return binary.hex()


binary_to_hex_udf = psf.udf(binary_to_hex, pst.StringType())

In [45]:
certs_df = spark.read.option(
    "basePath", base_path.format(year=ss_ts.year, month=ss_ts.month, day=ss_ts.day)
).format("parquet").load(
    base_path.format(year=ss_ts.year, month=ss_ts.month, day=ss_ts.day)
)

#withColumn("fp", binary_to_hex_udf(psf.col("fingerprint_sha256")))
moduli_df = certs_df.withColumn("moduli", binary_to_hex_udf(psf.col("parsed.subject_key_info.rsa.modulus"))
).select("moduli").distinct()

In [ ]:
timestamp = f"{ss_ts.year}{ss_ts.month:02}{ss_ts.day:02}"
output = f"luvizottocesarg-tmp/censys-certificates-v2-full-moduli-parquet/{timestamp}"
moduli_df.write.parquet(f"s3a://{output}")

In [51]:
moduli_df = spark.read.format("parquet").load(f"s3a://{output}")

In [52]:
moduli_df.count()

6262831123

In [59]:
goscanner_moduli_df = spark.read.format("parquet").load("s3a://luvizottocesarg-tmp/goscanner_moduli/20240709to12/unique_peer_certs_moduli.parquet")
all_moduli_df = moduli_df.unionByName(goscanner_moduli_df)

In [60]:
goscanner_moduli_df.count()

42839

In [58]:
all_moduli_df.distinct().count()

6262849496

In [ ]:
all_moduli_df.distinct().coalesce(10).write.option("compression","gzip").csv("s3a://luvizottocesarg-tmp/goscanner-censys-certificates-v2-full-moduli-csv")

In [ ]:
clean_spark()